In [ ]:
import pandas as pd
import numpy as np
import os
import sqlite3
from sqlalchemy import create_engine
import logging
import time
from ingestion_db import ingest_db


In [ ]:
conn = sqlite3.connect('mydatabase.db')

In [ ]:
tables=pd.read_sql_query("""
    SELECT name FROM sqlite_master WHERE type='table';
    """, conn)
tables

In [ ]:
for table in tables['name']:
    print('-'*50, f"{table}", '-'*50)
    print("count of records:", pd.read_sql_query(f"SELECT count(*) as count FROM {table}", conn)['count'].values[0])
    display(pd.read_sql_query(f"SELECT * FROM {table} limit 5", conn))

In [ ]:
purchases=pd.read_sql_query("""
    SELECT * FROM purchases where VendorNumber=388
    """, conn)
purchases

In [ ]:
purchase_prices=pd.read_sql_query("""
    SELECT * FROM purchase_prices where VendorNumber=388
    """, conn)
purchase_prices

In [ ]:
purchases.groupby(['Brand','PurchasePrice'])[['Quantity','Dollars']].sum()

In [ ]:

vendor_invoice=pd.read_sql_query("""
    SELECT * FROM vendor_invoice where VendorNumber=388
    """, conn)

In [ ]:
vendor_invoice.columns

In [ ]:
purchases

In [ ]:
purchases.groupby(['Brand','PurchasePrice'])[['Quantity','Dollars']].sum()

In [ ]:
# sales = pd.read_sql_query("SELECT * FROM sales", conn)
# sales.groupby(['Brand', 'SalesPrice'])[['SalesQuantity', 'SalesDollars']].sum()

In [ ]:
freight_summary = pd.read_sql_query("""
    SELECT VendorNumber, SUM(Freight) AS TotalFreight
    FROM vendor_invoice 
    GROUP BY VendorNumber
    """, conn)

In [44]:
pd.read_sql_query("""
    SELECT 
        p.VendorNumber,
        p.VendorName,
        p.Brand,
        p.PurchasePrice,
        pp.Price as Actual_price,
        pp.Volume,
        SUM(p.Quantity) AS TotalPurchaseQuantity,
        SUM(p.Dollars) AS Total_Purchase_Dollars
    FROM purchases p
    JOIN purchase_prices pp ON p.Brand = pp.Brand 
    where p.PurchasePrice > 0   
    GROUP BY 
        p.VendorNumber,
        p.VendorName,
        p.PurchasePrice,
        pp.Price,
        pp.Volume
    ORDER BY Total_Purchase_Dollars DESC
    """, conn)

,VendorNumber,VendorName,Brand,PurchasePrice,Actual_price,Volume,TotalPurchaseQuantity,Total_Purchase_Dollars
0,1128,BROWN-FORMAN CORP,1233,26.27,36.99,1750,145080,3811251.60
1,4425,MARTIGNETTI COMPANIES,3405,23.19,28.99,1750,164038,3804041.22
2,17035,PERNOD RICARD USA,8068,18.24,24.99,1750,187407,3418303.68
3,3960,DIAGEO NORTH AMERICA INC,4261,16.17,22.99,1750,201682,3261197.94
4,3960,DIAGEO NORTH AMERICA INC,3545,21.89,29.99,1750,138109,3023206.01
...,...,...,...,...,...,...,...,...
8747,3960,DIAGEO NORTH AMERICA INC,2626,1.42,1.99,50,2,2.84
8748,9815,WINE GROUP INC,8527,1.32,4.99,750,2,2.64
8749,8004,SAZERAC CO INC,5683,0.39,0.49,50,6,2.34
8750,3924,HEAVEN HILL DISTILLERIES,9123,0.74,0.99,50,2,1.48


In [45]:
pd.read_sql_query("""
    SELECT 
        VendorNo,
        Brand,
        SUM(SalesQuantity) AS Total_Sales_Quantity,
        SUM(SalesDollars) AS Total_Sales_Dollars,  
        SUM(SalesDollars) / SUM(SalesQuantity) AS SalesPrice
    FROM sales
    GROUP BY 
        VendorNo,
        Brand
    ORDER BY Total_Sales_Dollars DESC
    """, conn)

,VendorNo,Brand,Total_Sales_Quantity,Total_Sales_Dollars,SalesPrice
0,1128,1233,142049,5101919.51,35.916617
1,4425,3405,160247,4819073.49,30.072784
2,17035,8068,187140,4538120.60,24.249870
3,3960,4261,200412,4475972.88,22.333857
4,3960,3545,135838,4223107.62,31.089295
...,...,...,...,...,...
11267,3252,3933,2,1.98,0.990000
11268,3924,9123,2,1.98,0.990000
11269,10050,3623,2,1.98,0.990000
11270,9206,2773,1,0.99,0.990000


In [46]:
# start = time.time()
# final_table = pd.read_sql_query("""
# select 
#     PS.VendorNumber,
#     PS.Brand,
#     PS.PurchasePrice,
#     PS.Price as Actual_price,
#     sum(s.SalesQuantity) as Total_Sales_Quantity,
#     sum(s.SalesDollars) as Total_Sales_Dollars,
#     sum(s.SalesDollars) / sum(s.SalesQuantity) as SalesPrice,
#     sum(s.ExciseTax) as Total_Excise_Tax,
#     sum(v.Quantity) as Total_Purchase_Quantity,
#     sum(v.Dollars) as Total_Purchase_Dollars,
#     sum(v.Freight) as Total_Freight
#     from purchase_prices PS
#     join sales s on PS.Brand = s.Brand and PS.VendorNumber = s.VendorNo
#     join vendor_invoice v on PS.VendorNumber = v.VendorNumber
#     group by 
#         PS.VendorNumber,
#         PS.Brand,
#         PS.PurchasePrice,
#         PS.Price""", conn)
# end = time.time()

In [47]:
# start = time.time()

# final_table = pd.read_sql_query("""
#     SELECT 
#         pp.VendorNumber,
#         pp.Brand,
#         pp.PurchasePrice,
#         pp.Price AS Actual_price,
#         SUM(COALESCE(s.SalesQuantity, 0)) AS Total_Sales_Quantity,
#         SUM(COALESCE(s.SalesDollars, 0)) AS Total_Sales_Dollars,
#         CASE 
#             WHEN SUM(COALESCE(s.SalesQuantity, 0)) > 0 
#             THEN SUM(COALESCE(s.SalesDollars, 0)) / SUM(COALESCE(s.SalesQuantity, 0)) 
#             ELSE 0 
#         END AS SalesPrice,
#         SUM(COALESCE(s.ExciseTax, 0)) AS Total_Excise_Tax,
#         SUM(COALESCE(p.Quantity, 0)) AS Total_Purchase_Quantity,
#         SUM(COALESCE(p.Dollars, 0)) AS Total_Purchase_Dollars,
#         SUM(COALESCE(v.Freight, 0)) AS Total_Freight
#     FROM purchase_prices pp
#     LEFT JOIN sales s 
#         ON pp.Brand = s.Brand AND pp.VendorNumber = s.VendorNo
#     LEFT JOIN purchases p 
#         ON pp.Brand = p.Brand AND pp.VendorNumber = p.VendorNumber AND pp.PurchasePrice = p.PurchasePrice
#     LEFT JOIN vendor_invoice v 
#         ON pp.VendorNumber = v.VendorNumber
#     GROUP BY 
#         pp.VendorNumber,
#         pp.Brand,
#         pp.PurchasePrice,
#         pp.Price
# """, conn)

# end = time.time()


In [48]:

start = time.time()

vendor_summary = pd.read_sql_query("""
    WITH freight_summary AS (
        SELECT VendorNumber, SUM(Freight) AS TotalFreight
        FROM vendor_invoice
        GROUP BY VendorNumber
    ), 
    PURCHASE_SUMMARY AS (
        SELECT 
            p.VendorNumber,
            p.VendorName,
            p.Brand,
            p.Description,
            p.PurchasePrice,
            pp.Price as Actual_price,
            pp.Volume,
            SUM(p.Quantity) AS TotalPurchaseQuantity,
            SUM(p.Dollars) AS Total_Purchase_Dollars
        FROM purchases p
        JOIN purchase_prices pp 
            ON p.Brand = pp.Brand AND p.VendorNumber = pp.VendorNumber
        WHERE p.PurchasePrice > 0
        GROUP BY 
            p.VendorNumber,
            p.VendorName,
            p.Brand,
            p.Description,
            p.PurchasePrice,
            pp.Price,
            pp.Volume
    ),
    SALES_SUMMARY AS (
        SELECT 
            VendorNo,
            Brand,
            SUM(SalesQuantity) AS Total_Sales_Quantity,
            SUM(SalesDollars) AS Total_Sales_Dollars,
            SUM(SalesPrice) AS Total_Sales_Price,
            SUM(ExciseTax) AS Total_Excise_Tax 
        FROM sales
        GROUP BY VendorNo, Brand
    )
    SELECT 
        PS.VendorNumber,
        PS.VendorName,
        PS.Brand,
        PS.PurchasePrice,
        PS.Description,
        PS.Actual_price,
        PS.Volume,
        PS.TotalPurchaseQuantity,
        PS.Total_Purchase_Dollars,
        SS.Total_Sales_Quantity,
        SS.Total_Sales_Dollars,
        SS.Total_Sales_Price,
        SS.Total_Excise_Tax,
        FS.TotalFreight
    FROM PURCHASE_SUMMARY PS
    LEFT JOIN SALES_SUMMARY SS 
        ON PS.VendorNumber = SS.VendorNo AND PS.Brand = SS.Brand
    LEFT JOIN freight_summary FS 
        ON PS.VendorNumber = FS.VendorNumber
    ORDER BY Total_Purchase_Dollars DESC
""", conn)

end = time.time()

print(f"Query execution time: {end - start:.2f} seconds")



Query execution time: 63.89 seconds


In [49]:
vendor_summary.dtypes

VendorNumber                int64
VendorName                 object
Brand                       int64
PurchasePrice             float64
Description                object
Actual_price              float64
Volume                     object
TotalPurchaseQuantity       int64
Total_Purchase_Dollars    float64
Total_Sales_Quantity      float64
Total_Sales_Dollars       float64
Total_Sales_Price         float64
Total_Excise_Tax          float64
TotalFreight              float64
dtype: object

In [50]:
vendor_summary.isnull().sum()

VendorNumber                0
VendorName                  0
Brand                       0
PurchasePrice               0
Description                 0
Actual_price                0
Volume                      0
TotalPurchaseQuantity       0
Total_Purchase_Dollars      0
Total_Sales_Quantity      178
Total_Sales_Dollars       178
Total_Sales_Price         178
Total_Excise_Tax          178
TotalFreight                0
dtype: int64

In [51]:
vendor_summary["Description"].unique()

array(['Jack Daniels No 7 Black', "Tito's Handmade Vodka",
       'Absolut 80 Proof', ..., 'Crown Royal Apple',
       'Concannon Glen Ellen Wh Zin', 'The Club Strawbry Margarita'],
      shape=(9639,), dtype=object)

In [52]:
vendor_summary['Volume'] = vendor_summary['Volume'].astype('float64')

In [53]:
vendor_summary.fillna(0, inplace=True)

In [54]:
vendor_summary["VendorName"] = vendor_summary["VendorName"].str.strip()

In [56]:
vendor_summary["Profit"] = vendor_summary["Total_Sales_Dollars"] - vendor_summary["Total_Purchase_Dollars"]

vendor_summary["ROI"] = vendor_summary["Profit"] / vendor_summary["Total_Purchase_Dollars"]

vendor_summary["Profit_Margin"] = (vendor_summary["Profit"] / vendor_summary["Total_Sales_Dollars"]) * 100

vendor_summary["StockTurnover"] = vendor_summary["Total_Sales_Quantity"] - vendor_summary["TotalPurchaseQuantity"]

vendor_summary["Sales_Per_Purchase"] = vendor_summary["Total_Sales_Quantity"] / vendor_summary["TotalPurchaseQuantity"]


In [58]:
vendor_summary["Profit"].mean()
vendor_summary["Profit"].min()

np.float64(-52002.780000000006)

In [59]:
cursor = conn.cursor()

In [61]:
cursor.execute("DROP TABLE IF EXISTS vendor_summery;")
cursor.execute("""
    CREATE TABLE vendor_summery (
        VendorNumber int,
        VendorName varchar(100),    
        Brand int,
        Description varchar(100),
        PurchasePrice decimal(10,2),
        Actual_price decimal(10,2),
        Volume float,
        TotalPurchaseQuantity int,
        Total_Purchase_Dollars decimal(15,2),
        Total_Sales_Quantity decimal(15,2),
        Total_Sales_Dollars decimal(15,2),
        Total_Sales_Price decimal(15,2),
        Total_Excise_Tax decimal(15,2),
        TotalFreight decimal(15,2),
        Profit decimal(15,2),
        ROI decimal(15,6),
        Margin decimal(15,6),
        Profit_Margin decimal(15,6),
        StockTurnover decimal(15,2),
        Sales_Per_Purchase decimal(15,6),
        PRIMARY KEY (VendorNumber, Brand)
    );
    """)

In [62]:
pd.read_sql_query("SELECT * FROM vendor_summery", conn)

,VendorNumber,VendorName,Brand,Description,PurchasePrice,Actual_price,Volume,TotalPurchaseQuantity,Total_Purchase_Dollars,Total_Sales_Quantity,Total_Sales_Dollars,Total_Sales_Price,Total_Excise_Tax,TotalFreight,Profit,ROI,Margin,Profit_Margin,StockTurnover,Sales_Per_Purchase


In [63]:
vendor_summary.to_sql('vendor_summery', conn, if_exists='replace', index=False)

10648

In [64]:
vendor_summary

,VendorNumber,VendorName,Brand,PurchasePrice,Description,Actual_price,Volume,TotalPurchaseQuantity,Total_Purchase_Dollars,Total_Sales_Quantity,Total_Sales_Dollars,Total_Sales_Price,Total_Excise_Tax,TotalFreight,Profit,ROI,Profit_Margin,StockTurnover,Sales_Per_Purchase
0,1128,BROWN-FORMAN CORP,1233,26.27,Jack Daniels No 7 Black,36.99,1750.0,145080,3811251.60,142049.0,5101919.51,672819.31,260999.20,68601.68,1290667.91,0.338647,25.297693,-3031.0,0.979108
1,4425,MARTIGNETTI COMPANIES,3405,23.19,Tito's Handmade Vodka,28.99,1750.0,164038,3804041.22,160247.0,4819073.49,561512.37,294438.66,144929.24,1015032.27,0.266830,21.062810,-3791.0,0.976890
2,17035,PERNOD RICARD USA,8068,18.24,Absolut 80 Proof,24.99,1750.0,187407,3418303.68,187140.0,4538120.60,461140.15,343854.07,123780.22,1119816.92,0.327594,24.675786,-267.0,0.998575
3,3960,DIAGEO NORTH AMERICA INC,4261,16.17,Capt Morgan Spiced Rum,22.99,1750.0,201682,3261197.94,200412.0,4475972.88,420050.01,368242.80,257032.07,1214774.94,0.372493,27.139908,-1270.0,0.993703
4,3960,DIAGEO NORTH AMERICA INC,3545,21.89,Ketel One Vodka,29.99,1750.0,138109,3023206.01,135838.0,4223107.62,545778.28,249587.83,257032.07,1199901.61,0.396897,28.412764,-2271.0,0.983556
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10643,9815,WINE GROUP INC,8527,1.32,Concannon Glen Ellen Wh Zin,4.99,750.0,2,2.64,5.0,15.95,10.96,0.55,27100.41,13.31,5.041667,83.448276,3.0,2.500000
10644,8004,SAZERAC CO INC,5683,0.39,Dr McGillicuddy's Apple Pie,0.49,50.0,6,2.34,134.0,65.66,1.47,7.04,50293.62,63.32,27.059829,96.436186,128.0,22.333333
10645,3924,HEAVEN HILL DISTILLERIES,9123,0.74,Deep Eddy Vodka,0.99,50.0,2,1.48,2.0,1.98,0.99,0.10,14069.87,0.50,0.337838,25.252525,0.0,1.000000
10646,3960,DIAGEO NORTH AMERICA INC,6127,1.47,The Club Strawbry Margarita,1.99,200.0,1,1.47,72.0,143.28,77.61,15.12,257032.07,141.81,96.469388,98.974037,71.0,72.000000


In [66]:
vendor_summary.to_csv('vendor_summary(salses).csv', index=False)

In [ ]:
# def modify_data(df):
#     df['Volume'] = df['Volume'].astype('float64')
#     df["VendorName"] = df["VendorName"].str.strip()
#     df.fillna(0, inplace=True)

#     df["Profit"] = df["Total_Sales_Dollars"] - df["Total_Purchase_Dollars"]
#     df["ROI"] = (df["Profit"] / df["Total_Purchase_Dollars"]).replace([np.inf, -np.inf], 0)
#     df["Profit_Margin"] = ((df["Profit"] / df["Total_Sales_Dollars"]) * 100).replace([np.inf, -np.inf], 0)
#     df["StockTurnover"] = df["Total_Sales_Quantity"] - df["TotalPurchaseQuantity"]
#     df["Sales_Per_Purchase"] = (df["Total_Sales_Quantity"] / df["TotalPurchaseQuantity"]).replace([np.inf, -np.inf], 0)

#     return df

In [ ]:
# def create_vendor_summary_table(cursor):
#     cursor.execute("DROP TABLE IF EXISTS vendor_summery;")
#     cursor.execute("""
#     CREATE TABLE vendor_summery (
#         VendorNumber int,
#         VendorName varchar(100),    
#         Brand int,
#         Description varchar(100),
#         PurchasePrice decimal(10,2),
#         Actual_price decimal(10,2),
#         Volume float,
#         TotalPurchaseQuantity int,
#         Total_Purchase_Dollars decimal(15,2),
#         Total_Sales_Quantity decimal(15,2),
#         Total_Sales_Dollars decimal(15,2),
#         Total_Sales_Price decimal(15,2),
#         Total_Excise_Tax decimal(15,2),
#         TotalFreight decimal(15,2),
#         Profit decimal(15,2),
#         ROI decimal(15,6),
#         Margin decimal(15,6),
#         Profit_Margin decimal(15,6),
#         StockTurnover decimal(15,2),
#         Sales_Per_Purchase decimal(15,6),
#         PRIMARY KEY (VendorNumber, Brand)
#     );
#     """)